In [1]:
dados ='C:/Users/matto/Desktop/Leandro/Mestrado PPCIC/Dissertação/novo tema/Novo cj de dados/Conv1d/Dominio do Tempo/Todos os arrays/dados_tempo.npy'
rotulos ='C:/Users/matto/Desktop/Leandro/Mestrado PPCIC/Dissertação/novo tema/Novo cj de dados/rotulos.npy'

In [2]:
import numpy as np
X = np.load(dados)
y = np.load(rotulos)

In [3]:
from sklearn.model_selection import train_test_split

# Dividindo os dados em conjunto de treinamento e teste
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)



In [4]:
X_train.shape

(1493, 400, 1)

In [5]:
X_val.shape

(641, 400, 1)

In [6]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)

y_val = to_categorical(y_val)

In [7]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, Flatten, MaxPooling1D
import optuna

def objective(trial):
    # Definindo o espaço de busca dos hiperparâmetros
    filter_size = trial.suggest_int('filter_size', 8, 600, log=True)
    kernel_size = trial.suggest_int('kernel_size', 2, 6)
    num_layers = trial.suggest_int('num_layers', 2, 4)
    dense_neurons = trial.suggest_int('dense_neurons', 60, 320, log=True)
    max_pool = 2
    dropout = trial.suggest_int('dropout', 2, 5, step=1) / 10.0
    learning_rate = trial.suggest_categorical('learning_rate', [0.0001, 0.0003,0.0006,0.001,0.003,0.006, 0.01])

    model = keras.Sequential()

    for i in range(num_layers):
        if i == 0:
            model.add(Conv1D(filters=filter_size, kernel_size=kernel_size, activation="relu", input_shape=(400, 1)))
            model.add(MaxPooling1D(pool_size=max_pool))
            model.add(Dropout(dropout))
        else:
            if filter_size < kernel_size:
                filter_size = kernel_size
            filter_size *= 2
            model.add(Conv1D(filters=filter_size, kernel_size=kernel_size, activation="relu"))
            model.add(MaxPooling1D(pool_size=max_pool))
            model.add(Dropout(dropout))

    model.add(Flatten())
    model.add(Dense(dense_neurons, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    historico = model.fit(X_train, y_train, batch_size=32, epochs=25, validation_data=(X_val, y_val), verbose=1)

    return historico.history['val_accuracy'][-1]




In [8]:
import csv

# criação do objeto de estudo
study = optuna.create_study(direction="maximize")

# abertura do arquivo CSV em modo de escrita
with open("results.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile, delimiter=",")
    writer.writerow(["trial", "accuracy", "hyperparameters"])  # cabeçalho

    # iteração dos trials
    
    study.optimize(objective, n_trials=80)
    best_trial = study.best_trial
    print("Trial {}: Accuracy = {}".format(i, best_trial.value))

    # escrita dos valores no arquivo CSV
    writer.writerow([i, best_trial.value, best_trial.params])

    print("Trial {}: Accuracy = {}".format(i, best_trial.value))

print(study.best_params)

[I 2023-07-07 15:42:05,469] A new study created in memory with name: no-name-0f19ef0b-b0e8-4887-8088-2e77d9113ece


Epoch 1/25
47/47 [==============================] - 6s 99ms/step - loss: 0.4088 - accuracy: 0.8573 - val_loss: 0.5372 - val_accuracy: 0.8612
Epoch 2/25
47/47 [==============================] - 4s 87ms/step - loss: 0.3394 - accuracy: 0.8667 - val_loss: 0.3748 - val_accuracy: 0.8612
Epoch 3/25
47/47 [==============================] - 4s 87ms/step - loss: 0.2470 - accuracy: 0.9035 - val_loss: 0.3024 - val_accuracy: 0.9579
Epoch 4/25
47/47 [==============================] - 4s 86ms/step - loss: 0.1826 - accuracy: 0.9277 - val_loss: 0.3283 - val_accuracy: 0.9485
Epoch 5/25
47/47 [==============================] - 4s 83ms/step - loss: 0.1698 - accuracy: 0.9545 - val_loss: 0.3501 - val_accuracy: 0.9407
Epoch 6/25
47/47 [==============================] - 4s 82ms/step - loss: 0.1654 - accuracy: 0.9498 - val_loss: 0.2899 - val_accuracy: 0.9579
Epoch 7/25
47/47 [==============================] - 4s 87ms/step - loss: 0.1463 - accuracy: 0.9618 - val_loss: 0.2588 - val_accuracy: 0.9532
Epoch 8/25
47

[I 2023-07-07 15:44:04,372] Trial 0 finished with value: 0.9812792539596558 and parameters: {'filter_size': 73, 'kernel_size': 5, 'num_layers': 3, 'dense_neurons': 65, 'dropout': 5, 'learning_rate': 0.0006}. Best is trial 0 with value: 0.9812792539596558.


Epoch 1/25
47/47 [==============================] - 2s 24ms/step - loss: 0.3932 - accuracy: 0.8667 - val_loss: 0.4079 - val_accuracy: 0.8612
Epoch 2/25
47/47 [==============================] - 1s 21ms/step - loss: 0.3812 - accuracy: 0.8667 - val_loss: 0.3833 - val_accuracy: 0.8612
Epoch 3/25
47/47 [==============================] - 1s 21ms/step - loss: 0.3694 - accuracy: 0.8667 - val_loss: 0.4182 - val_accuracy: 0.8612
Epoch 4/25
47/47 [==============================] - 1s 22ms/step - loss: 0.3535 - accuracy: 0.8667 - val_loss: 0.4092 - val_accuracy: 0.8612
Epoch 5/25
47/47 [==============================] - 1s 21ms/step - loss: 0.3378 - accuracy: 0.8667 - val_loss: 0.3965 - val_accuracy: 0.8861
Epoch 6/25
47/47 [==============================] - 1s 22ms/step - loss: 0.3236 - accuracy: 0.8694 - val_loss: 0.3779 - val_accuracy: 0.9251
Epoch 7/25
47/47 [==============================] - 1s 21ms/step - loss: 0.3102 - accuracy: 0.8734 - val_loss: 0.3106 - val_accuracy: 0.9204
Epoch 8/25
47

[I 2023-07-07 15:44:31,346] Trial 1 finished with value: 0.9578782916069031 and parameters: {'filter_size': 17, 'kernel_size': 4, 'num_layers': 2, 'dense_neurons': 159, 'dropout': 3, 'learning_rate': 0.0001}. Best is trial 0 with value: 0.9812792539596558.


...

Epoch 1/25
47/47 [==============================] - 178s 4s/step - loss: 0.4143 - accuracy: 0.8620 - val_loss: 0.4740 - val_accuracy: 0.8612
Epoch 2/25
47/47 [==============================] - 175s 4s/step - loss: 0.3513 - accuracy: 0.8667 - val_loss: 0.4135 - val_accuracy: 0.8612
Epoch 3/25
47/47 [==============================] - 176s 4s/step - loss: 0.2017 - accuracy: 0.9176 - val_loss: 0.2579 - val_accuracy: 0.9657
Epoch 4/25
47/47 [==============================] - 177s 4s/step - loss: 0.1370 - accuracy: 0.9645 - val_loss: 0.1568 - val_accuracy: 0.9704
Epoch 5/25
47/47 [==============================] - 175s 4s/step - loss: 0.1072 - accuracy: 0.9725 - val_loss: 0.1676 - val_accuracy: 0.9641
Epoch 6/25
47/47 [==============================] - 157s 3s/step - loss: 0.0953 - accuracy: 0.9752 - val_loss: 0.1386 - val_accuracy: 0.9657
Epoch 7/25
47/47 [==============================] - 103s 2s/step - loss: 0.1018 - accuracy: 0.9705 - val_loss: 0.2399 - val_accuracy: 0.9454
Epoch 8/25
47

[I 2023-07-08 12:58:50,981] Trial 78 finished with value: 0.9719188809394836 and parameters: {'filter_size': 314, 'kernel_size': 3, 'num_layers': 4, 'dense_neurons': 291, 'dropout': 2, 'learning_rate': 0.0001}. Best is trial 2 with value: 0.9968798756599426.


Epoch 1/25
47/47 [==============================] - 10s 184ms/step - loss: 0.4781 - accuracy: 0.8553 - val_loss: 0.4013 - val_accuracy: 0.8612
Epoch 2/25
47/47 [==============================] - 8s 179ms/step - loss: 0.3714 - accuracy: 0.8667 - val_loss: 0.3742 - val_accuracy: 0.8612
Epoch 3/25
47/47 [==============================] - 8s 179ms/step - loss: 0.2111 - accuracy: 0.8667 - val_loss: 0.2825 - val_accuracy: 0.8612
Epoch 4/25
47/47 [==============================] - 8s 179ms/step - loss: 0.2076 - accuracy: 0.9102 - val_loss: 0.2417 - val_accuracy: 0.9704
Epoch 5/25
47/47 [==============================] - 8s 179ms/step - loss: 0.1661 - accuracy: 0.9491 - val_loss: 0.2192 - val_accuracy: 0.9626
Epoch 6/25
47/47 [==============================] - 9s 184ms/step - loss: 0.1301 - accuracy: 0.9685 - val_loss: 0.1854 - val_accuracy: 0.9688
Epoch 7/25
47/47 [==============================] - 8s 181ms/step - loss: 0.1399 - accuracy: 0.9618 - val_loss: 0.1273 - val_accuracy: 0.9719
Epoch

[I 2023-07-08 13:02:26,909] Trial 79 finished with value: 0.9953197836875916 and parameters: {'filter_size': 70, 'kernel_size': 5, 'num_layers': 4, 'dense_neurons': 261, 'dropout': 4, 'learning_rate': 0.001}. Best is trial 2 with value: 0.9968798756599426.


NameError: name 'i' is not defined